The data was from https://warwick.ac.uk/fac/cross_fac/tia/data/glascontest/, which was originally used for MICCAI Glas challenge.

In [2]:
import pooch
from skimage.io import imread
from aicsimageio.writers import OmeTiffWriter
import zipfile
from pathlib import Path
from random import random
import numpy as np

data_path = Path("../../data/semantic2D")
p = data_path / Path("download")
p.mkdir(exist_ok=True, parents=True)
p = data_path / Path("train")
p.mkdir(exist_ok=True)
p = data_path / Path("testA")
p.mkdir(exist_ok=True)
p = data_path / Path("testB")
p.mkdir(exist_ok=True)

In [3]:
source = pooch.retrieve(
    url="https://warwick.ac.uk/fac/cross_fac/tia/data/glascontest/download/warwick_qu_dataset_released_2016_07_08.zip",
    fname="source.zip",
    path=data_path / Path("download"),
    known_hash="7849d349db105751a157d011e9708d724203e85bbad08fd047dcdfe5b9a572e2"
)

In [4]:
# unzip the data
with zipfile.ZipFile(source,"r") as zip_ref:
    zip_ref.extractall(data_path / Path("download"))

In [5]:
source_path = data_path / Path("download") / Path("Warwick QU Dataset (Released 2016_07_08)")

In [6]:
# parameters about the dataset
num_train = 85
num_testA = 60
num_testB = 20

dataset_min_size = 10000000

for idx in range(num_train):
    im = imread(source_path / f"train_{idx+1}.bmp")
    gt = imread(source_path / f"train_{idx+1}_anno.bmp")

    # convert to binary
    gt[gt > 0] = 1

    # save
    out_fn = data_path / Path("train") / f"img_{idx:02d}_IM.tiff"
    OmeTiffWriter.save(im, out_fn, dim_order="YXS")

    out_fn = data_path / Path("train") / f"img_{idx:02d}_GT.tiff"
    OmeTiffWriter.save(gt, out_fn, dim_order="YX")

    # check size
    if gt.shape[0] < dataset_min_size:
        dataset_min_size = gt.shape[0]
    if gt.shape[1] < dataset_min_size:
        dataset_min_size = gt.shape[1]

for idx in range(num_testA):
    im = imread(source_path / f"testA_{idx+1}.bmp")
    gt = imread(source_path / f"testA_{idx+1}_anno.bmp")

    # convert to binary
    gt[gt > 0] = 1

    # save
    out_fn = data_path / Path("testA") / f"img_{idx:02d}_IM.tiff"
    OmeTiffWriter.save(im, out_fn, dim_order="YXS")

    out_fn = data_path / Path("testA") / f"img_{idx:02d}_GT.tiff"
    OmeTiffWriter.save(gt, out_fn, dim_order="YX")

    # check size
    if gt.shape[0] < dataset_min_size:
        dataset_min_size = gt.shape[0]
    if gt.shape[1] < dataset_min_size:
        dataset_min_size = gt.shape[1]

for idx in range(num_testB):
    im = imread(source_path / f"testB_{idx+1}.bmp")
    gt = imread(source_path / f"testB_{idx+1}_anno.bmp")

    # convert binary
    gt[gt > 0] = 1

    # save
    out_fn = data_path / Path("testB") / f"img_{idx:02d}_IM.tiff"
    OmeTiffWriter.save(im, out_fn, dim_order="YXS")

    out_fn = data_path / Path("testB") / f"img_{idx:02d}_GT.tiff"
    OmeTiffWriter.save(gt, out_fn, dim_order="YX")

    # check size
    if gt.shape[0] < dataset_min_size:
        dataset_min_size = gt.shape[0]
    if gt.shape[1] < dataset_min_size:
        dataset_min_size = gt.shape[1]


/mnt/eternus/anaconda3_shared/envs/jc_im2im_dev/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [7]:
print(f"the patch size during training should not be smaller than {dataset_min_size} or pad to be large enough")

the patch size during training should not be smaller than 430 or pad to be large enough


In [8]:
# you may remove the download folder now.
from shutil import rmtree
rmtree(data_path / Path("download"))